In [ ]:
import os
import numpy as np
from scipy.io import loadmat
from sparse_coder import SparseCoder, get_effective_dimensionality
import cPickle as pkl
import matplotlib.pyplot as plt
%matplotlib inline
import sys
from prep_field_dataset import get_data_matrix
sys.path.append('..')
from utils.rf_plot import show_fields

In [ ]:
data = get_data_matrix(l_patch=20)
d_eff = get_effective_dimensionality(data)

In [ ]:
output_dir = 'output/'
fn_ = os.listdir(output_dir)
fn_ = [os.path.join(output_dir, fn) for fn in fn_ if fn.endswith('pkl')]
fn_.sort()

In [ ]:
fn_

In [ ]:
summary_ = []
for fn in fn_:
    with open(fn, 'rb') as f:
        res = pkl.load(f)
    summ = (fn, res['D'])
    summary_.append(summ)

In [ ]:
for fn, D in summary_:
    print fn, D.shape

In [ ]:
fn, D = summary_[1]
plt.figure(figsize=(18, 16))
plt.title(fn)
show_fields(D)

In [ ]:
sc = SparseCoder.restore(data, fn_[2])

In [ ]:
i_idx = sc.train(n_itr=1, eta=0.0, cost_list=[], n_g_itr=300)

In [ ]:
self = sc
A = self.tc.get_coefficients()
D = self.tc.get_dictionary()
Ih = np.dot(A, D)
I = self.tc.t_DATA.get_value()[i_idx]

In [ ]:
signal = np.sqrt((I ** 2).sum(axis=1))
error = np.sqrt(((I - Ih) ** 2).sum(axis=1))
plt.scatter(signal, (signal/error) ** 2)
plt.xlabel('Signal')
plt.ylabel('SNR') 
# plt.xlim([0, 15])
# plt.ylim([0, 3.5])

In [ ]:
plt.hist((I ** 2).sum(axis=1) / ((I - Ih) ** 2).sum(axis=1), bins=50);

In [ ]:
sc.plot_example(4, i_idx, 20)

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(np.log(evals[::-1])[0:-8])

plt.subplot(1, 2, 2)
plt.title('Percent variance explained as a number of eigenvectors')

plt.plot(cs / cs[-1])
plt.plot(np.arange(len(cs)), np.zeros(len(cs)) + 0.95)

In [ ]:
patch = data[4].reshape(l_patch, l_patch)
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.imshow(patch, cmap=plt.cm.gray)
plt.subplot(1, 2, 2)
plt.imshow(np.abs(np.fft.fftshift(np.fft.fft2(patch))), cmap=plt.cm.gray, interpolation='nearest')
plt.colorbar()